In [1]:
# n jobs for parallel tasks
N_JOBS = -1 

In [2]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm_notebook as tqdm
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from draw_place import DrawPlace

from sklearn.cluster import KMeans

In [3]:
db = pickle.load(open( "db.pkl", "rb" ))

In [4]:
placement_hist = pickle.load(open("plot_hist.pkl","rb"))

In [5]:
TXY = [np.stack(placement_hist[i]).T for i in tqdm(range(len(placement_hist)))]

In [6]:
def split_nodes(i):
    return np.stack([TXY[j][i] for j in range(len(TXY))]).astype('float64')
traj_list = Parallel(n_jobs=N_JOBS, verbose=1, backend="threading")(delayed(split_nodes)(i) for i in range(TXY[0].shape[0]))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 11234 tasks  

In [7]:
X_d = np.stack([x.flatten() for x in tqdm(traj_list)])

In [37]:

branch_factor = 100
levels = 2
noise_tolerance = 1000
"""
cluster_mat = np.zeros((X_d.shape[0],levels))

idx_map = np.zeros((X_d.shape[0],levels))

X_d_list = [(X_d,)]
for i in tqdm(range(levels)):
    X_d_list_tmp = []
    for X_di, imap in X_d_list:
        k_means = KMeans(branch_factor, n_jobs = N_JOBS).fit(X_di)
        labels = np.unique(k_means.labels_)
        for label in labels.tolist():
            l_idx = np.where(k_means.labels_ == label)
            cluster_mat[i,imap][l_idx] = label
            X_d_list_tmp.append(X_d[i][l_idx])
    X_d_list = X_d_list_tmp
"""

import numpy as np
from scipy.cluster.vq import *
import pickle


class tree(object):
	def __init__(self, name, data=None, additional_data=None, children=None):
		self.name = name
		self.data = data
		self.additional_data = additional_data
		if children is None:
			self.children = []
		else:
			self.children = children

	def set_data(self, data):
		self.data = data

	def get_data(self):
		return self.data

	def set_additional_data(self, additional_data):
		self.additional_data = additional_data

	def get_additional_data(self):
		return self.additional_data
    
	def set_name(self, name):
		self.name = name

	def get_name(self):
		return self.name

	def set_children(self, children):
		self.children = children

	def add_child(self, child):
		self.children.append(child)

	def add_children(self, children):
		self.children.extend(children)

	def get_children_number(self):
		return len(self.children)

	def gather_data(self, gather_additional_data=False):
		'''Gather data from its children, depth=1'''
		if gather_additional_data:
			return [child.get_additional_data() for child in self.children]
		else:
			return [child.get_data() for child in self.children]

	def find(self, data, find_additional_data=False):
		if find_additional_data:
			if self.additional_data == data:
				return self
			else:
				for child in self.children:
					res = child.find(data, find_additional_data)
					if res != -1:
						return res
				return -1
		else:
			if self.data == data:
				return self
			else:
				for child in self.children:
					res = child.find(data, find_additional_data)
					if res != -1:
						return res
				return -1

	def is_leaf_node(self):
		return len(self.children) == 0

	def gather_leaves(self, output):
		assert type(output) == list, 'output must be a list.'
		if self.is_leaf_node():
			output.append(self)
			return
		for child in self.children:
			child.gather_leaves(output)

	def gather_data_from_leaves(self, output, gather_additional_data=False):
		assert type(output) == list, 'output must be a list.'
		if gather_additional_data:
			if self.is_leaf_node():
				output.append(self.additional_data)
				return
			for child in self.children:
				child.gather_data_from_leaves(output, gather_additional_data)
		else:
			if self.is_leaf_node():
				output.append(self.data)
				return
			for child in self.children:
				child.gather_data_from_leaves(output, gather_additional_data)

	def __str__(self):
		return str(self.name)

	def __repr__(self):
		return str(self)

	def __iter__(self):
		return iter(self.children)

	def __hash__(self):
		return hash(str(self.name))


class hierarchical_kmeans(object):
	def __init__(self, clusters):
		self.clusters = clusters  # tree type variable
		self.root = tree('root')
		self.total_k = 0

	def cluster(self, data, only_store_id=True, iteration=1):
		data = np.asarray(data)
		self.only_store_id = only_store_id
		cluster = self.clusters
		current_node = self.root
		idx = np.arange(data.shape[0])
		print('Doing hierarchical kmeans...')
		self._cluster(data, idx, current_node, cluster, iteration=iteration, only_store_id=only_store_id)
		print('Done! Actual total number of clusters is %d.' % self.total_k)

	def _cluster(self, data, idx, current_node, cluster_node, iteration=1, only_store_id=True):
		if cluster_node.is_leaf_node():
			self.total_k += 1
			if only_store_id:
				current_node.set_additional_data(idx)
			else:
				current_node.set_additional_data(data[idx])
			return

		codebook, _ = kmeans(data[idx], cluster_node.get_children_number(), iter=iteration)
		ids, _ = vq(data[idx], codebook)

		if codebook.shape[0] == 1:  # kmeans only find one cluster
			current_node.set_data(codebook[0])
			if only_store_id:
				current_node.set_additional_data(idx)
			else:
				current_node.set_additional_data(data[idx])
			return

		for i, c in enumerate(cluster_node):
			if i >= codebook.shape[0]:  # cluster number smaller than the number of next branch
				break
			idx_i = idx[np.nonzero(ids == i)[0]]
			branch = tree(c.get_name(), codebook[i])
			current_node.add_child(branch)
			self._cluster(data, idx_i, branch, c, iteration=iteration, only_store_id=only_store_id)

	def save(self, file_name):
		with open(file_name, 'wb') as f:
			pickle.dump([self.clusters, self.root, self.total_k], f)

	@staticmethod
	def load(file_name):
		hk = hierarchical_kmeans(None)
		with open(file_name, 'rb') as f:
			[hk.clusters, hk.root, hk.total_k] = pickle.load(f)
		return hk
    
	def prune(self, eps, max_depth=-1):
		self._prune(eps, self.root, 0, max_depth)
    
	def _prune(self, eps, current_node, current_depth, max_depth=-1):
		if current_node.is_leaf_node():
			codebook = current_node.gather_data()
			current_depth += 1
			if len(codebook) < eps:
				current_node.set_name('noise')
				current_node.set_children([])
        
		for i, c in enumerate(current_node):
			self._prune(eps, c, current_depth, max_depth)

	def find_cluster(self, data, max_depth=-1):
		data = np.asarray(data)
		if len(data.shape) == 1:
			return self._find_cluster([data], self.root, 0, max_depth)
		return [self._find_cluster([d], self.root, 0, max_depth) for d in data]

	def _find_cluster(self, data, current_node, current_depth, max_depth=-1):
		if current_node.is_leaf_node() or (max_depth >= current_depth and max_depth > 0):
			return current_node
		codebook = current_node.gather_data()
		id, _ = vq(data, codebook)
		current_depth += 1
		branch = current_node.children[id[0]]
		return self._find_cluster(data, branch, current_depth, max_depth)

In [38]:
X_d.shape

(254457, 120)

In [39]:
cluster_structure = tree('clust_test')
for i in range(branch_factor):
    x = tree('l1-c%d' % (i+1))
    cluster_structure.add_child(x)
    for j in range(10):
        y = tree('l2-c%d-c%d' % (i+1, j+1))
        x.add_child(y)
hk = hierarchical_kmeans(cluster_structure)
hk.cluster(X_d, only_store_id=True, iteration=1)

Doing hierarchical kmeans...
Done! Actual total number of clusters is 999.


In [40]:
hk.prune(5)

In [41]:
hk.root.children[0].children[0].name

'noise'

In [42]:
#hk.find_cluster(X_d[0,:])
lab = [hk.find_cluster(X_d[i,:]).get_name() for i in tqdm(range(X_d.shape[0]))]

In [45]:
'noise' in lab

True

In [44]:
len([x for x in lab if x == 'noise']), len(lab)

(254457, 254457)

In [ ]:
id2idx = list(set(lab))
labid = [id2idx.index(x) for x in tqdm(lab)]

In [ ]:
labid = np.array(labid)

In [ ]:
np.where(labid == 1)

In [ ]:
np.where(k_means.labels_ == 1)

In [ ]:
from draw_place import DrawPlace
d = DrawPlace(db)
#clust2col = [list(np.random.choice(range(256), size=3)/255) for _ in range(np.unique(k_means.labels_).shape[0])]

#for i in tqdm(range(len(placement_hist))):
#    d.forward(np.concatenate([placement_hist[i][0],placement_hist[i][1]]), (k_means.labels_).astype(np.int), clust2col, 'plots/plot_'+str(i)+str('.png'))
clust2col = [list(np.random.choice(range(256), size=3)/255) for _ in range(np.unique(labid).shape[0])]

for i in tqdm(range(len(placement_hist))):
    d.forward(np.concatenate([placement_hist[i][0],placement_hist[i][1]]), (labid).astype(np.int), clust2col, 'plots/'+str(i)+str('.png'))